<a href="https://colab.research.google.com/github/periwinklemctartins/Coronavirus--Propogation-Simulation/blob/master/VectorEquations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import torch
import matplotlib.pyplot as plt
import matplotlib
import numpy as np

In [0]:
def test(yo):

  yo += 2

  a = torch.tensor([1,2])
  b = torch.tensor([[3,4],[5,6]])

  return a, b




In [17]:
a = torch.tensor([1,2])
b = torch.tensor([1,2])

c = a + b

print(c)

tensor([2, 4])


## Variable Initialization

In [0]:
# province = [abrev, population, # of initial infections, time between first case in Canada and first case in province]

AB = ['AB', 4413146, 1, 37]
BC = ['BC', 5110917, 1, 0]
MB = ['MB', 1377517, 1, 42]
NB = ['NB', 779993, 1, 42]
NL = ['NL', 521365, 1, 48]
NT = ['NT', 44904, 1, 50]
NS = ['NS', 977457, 1, 47]
NU = ['NU', 39097, 0, 0]
ON = ['ON', 14711827, 3, 0]
PE = ['PE', 158158, 1, 44]
QC = ['QC', 8537674, 1, 30]
SK = ['SK', 1181666, 1, 45]
YT = ['YT', 41078, 2, 52]

Provinces = [ AB, BC, MB, NB, NL, NT, NS, NU, ON, PE, QC, SK, YT ]

L = torch.tensor([[6,    -1,    -1,     0,     0,     0,     0,     0,    -1,     0,    -1,    -1,    -1], 
                  [-1,    6,    -1,     0,     0,    -1,     0,     0,    -1,     0,    -1,    -1,     0],
                  [-1,    1,     7,     0,     0,    -1,     0,    -1,    -1,     0,    -1,    -1,     0],
                  [0,     0,     0,     4,    -1,     0,     0,     0,    -1,    -1,    -1,     0,     0],
                  [0,     0,     0,    -1,     5,     0,    -1,     0,    -1,    -1,    -1,     0,     0],
                  [0,    -1,    -1,     0,     0,     7,     0,    -1,    -1,     0,    -1,    -1,    -1],
                  [0,     0,     0,     0,    -1,     0,     4,     0,    -1,    -1,    -1,     0,     0],
                  [0,     0,    -1,     0,     0,    -1,     0,     6,    -1,     0,    -1,    -1,    -1],
                  [-1,    -1,    -1,    -1,    -1,    -1,    -1,    -1,    12,    -1,    -1,    -1,   -1],
                  [0,     0,     0,    -1,    -1,     0,    -1,     0,    -1,     5,    -1,     0,     0],
                  [-1,    -1,    -1,    -1,    -1,    -1,    -1,    -1,    -1,    -1,    12,    -1,   -1],
                  [-1,    -1,    -1,     0,     0,    -1,     0,    -1,    -1,     0,    -1,     7,    0],
                  [-1,     0,     0,     0,     0,    -1,     0,    -1,    -1,     0,    -1,     0,    5]]);

In [0]:
S =  torch.zeros(13)
E =  torch.zeros(13)
I =  torch.zeros(13)
S0 = torch.zeros(13)
E0 = torch.zeros(13)
I0 = torch.zeros(13)

FE = 0.01     # Fraction of population that is initially exposed

# Initializes Susceptible
for n in range(13):
    Province = Provinces[n]
    
    # Initializes Susceptible
    S0[n] = Province[1]*(1 - FE)

    # Initializes Exposed
    E0[n] = Province[1]*FE

In [32]:
dt = 0.05; nt = 4000

alpha = 0.2
beta  = 0.5
gamma = 0.01
mu    = 0.075
kE    = 0.0005
kS    = 0.0005
kI    = 0
theta = torch.tensor([alpha, beta, gamma, mu, kE, kS, kI])

P = ['AB', 'BC', 'MB', 'NB', 'NL', 'NT', 'NS', 'NU', 'ON', 'PE', 'QC', 'SK', 'YT']

Sout, Eout, Iout = integrateSEI(theta,S0,E0,I0,dt,nt)

S = torch.zeros(nt+1); E = torch.zeros(nt+1); I = torch.zeros(nt+1)

for n in range(13):
  for i in range(nt+1):
    S[i] = Sout[n,i]; E[i] = Eout[n,i]; I[i] = Iout[n,i]
  
  t = np.arange(nt+1)*dt
  title = 'maximum of infected people %', P[n], # torch.max(I).item()
  plt.figure(n)
  plt.title(title)
  plt.plot(t,S,t,E,t,I)

RuntimeError: ignored

In [0]:
# First define the right hand side, we store all the parameters
# in a single vector theta = [alpha, beta, gamma, mu]
def SEImodel(theta,S,E,I):
    
    alpha = theta[0]
    beta  = theta[1]
    gamma = theta[2]
    mu    = theta[3]
    kE    = theta[4]
    kS    = theta[5]
    kI    = theta[6]

    # dSdt = tensor.zeros(13)
    # dEdt = tensor.zeros(13)
    # dIdt = tensor.zeros(13)

    print(S)
    print(E)
    print(I)

    dSdt  = -kS*torch.mv(L,S) - beta*(E*S) - gamma*(I*S)                # dS/dt
    dEdt  = -kE*torch.mv(L,E) + beta*(E*S)  + gamma*(I*S) - alpha*E      # dE/dt
    dIdt  = -kI*torch.mv(L,I) + alpha*E - mu*I                       # dI/dt
    
    return dSdt, dEdt, dIdt

In [57]:
S =  torch.zeros(13)
E =  torch.zeros(13)
I =  torch.zeros(13)
S0 = torch.zeros(13)
E0 = torch.zeros(13)
I0 = torch.zeros(13)

FE = 0.01     # Fraction of population that is initially exposed

# Initializes Susceptible
for n in range(13):
    Province = Provinces[n]
    
    # Initializes Susceptible
    S[n] = int(round(Province[1]*(1 - FE)))

    # Initializes Exposed
    E[n] = Province[1]*FE

alpha = 0.2
beta  = 0.5
gamma = 0.01
mu    = 0.075
kE    = 1
kS    = 1
kI    = 0
theta = torch.tensor([alpha, beta, gamma, mu, kE, kS, kI])

dSdt  = torch.mv(L,S) - beta*(E*S) - gamma*(I*S)

print(S)

RuntimeError: ignored

In [41]:
M = torch.tensor([[1,2],[3,4]])
V = torch.tensor([5,6])

p = torch.mv(M,V)

print(p)

tensor([17, 39])


In [0]:
def integrateSEI(theta,S0,E0,I0,dt,nt):
    
    # tensors to save the results over time
    Sout = torch.zeros(13,nt+1)
    Eout = torch.zeros(13,nt+1)
    Iout = torch.zeros(13,nt+1)
    
    for i in range(13):
        Sout[i,0] = S0[i]
        Eout[i,0] = E0[i]
        Iout[i,0] = E0[i]
    
    # Sout = torch.zeros(nt+1); Sout[0] = S0
    # Eout = torch.zeros(nt+1); Eout[0] = E0
    # Iout = torch.zeros(nt+1); Iout[0] = I0
    
    S = S0; E = E0; I = I0
    for i in range(nt):
        
        dSdt, dEdt, dIdt = SEImodel(theta,S,E,I)
        S += dt*dSdt
        E += dt*dEdt
        I += dt*dIdt
        
        for n in range(13):
            Sout[n,i+1] = S[n]; Eout[n,i+1] = E[n]; Iout[n,i+1] = I[n]
        
    return Sout, Eout, Iout